# Pdf Form Filler

## Form Field Text Update

### Imput pdf Paths

In [31]:
# input_path should contain a pdf file with fields to fill
# output_path creates the pdf file with the fields fullfiled
# Create the pdf file with form fields with: https://www.pdfescape.com
input_path = 'pdf_to_fill_pdfescape.pdf'
output_path = 'pdf_filled.pdf'

### Get Dictionary of fields to replace

In [32]:
from pdfrw import PdfReader
def get_pdf_info(path):
    pdf = PdfReader(path)
    annotations = pdf.pages[0]['/Annots']
    fields = {}
    i=1
    for annotation in annotations:
        field = (annotation.T).replace('(','').replace(')','')
        #print(field)
        fields[str(field)] = str(i)
        i+=1
    print(fields)
    
if __name__ == '__main__':
    get_pdf_info(input_path)

{'field_1': '1', 'field_2': '2', 'field_3': '3'}


### Input the dictionary of field names with its content

In [33]:
data_dict = {
   'field_1': 'This is Field 1',
   'field_2': 'This is Field 2',
   'field_3': 'This is Field 3'
}

### Function to fill the pdf Form

In [34]:
# Define dependencies
import os
import pdfrw

ANNOT_KEY = '/Annots'
ANNOT_FIELD_KEY = '/T'
ANNOT_VAL_KEY = '/V'
ANNOT_RECT_KEY = '/Rect'
SUBTYPE_KEY = '/Subtype'
WIDGET_SUBTYPE_KEY = '/Widget'

def write_fillable_pdf(input_pdf_path, output_pdf_path, data_dict):
    template_pdf = pdfrw.PdfReader(input_pdf_path)
    # Makes it render when opening the pdf file on Adobe Acrobat DC
    # Credit @TLK3: https://github.com/pmaupin/pdfrw/issues/84
    template_pdf.Root.AcroForm.update(pdfrw.PdfDict(NeedAppearances=pdfrw.PdfObject('true')))     
    annotations = template_pdf.pages[0][ANNOT_KEY]
    for annotation in annotations:
        if annotation[SUBTYPE_KEY] == WIDGET_SUBTYPE_KEY:
            if annotation[ANNOT_FIELD_KEY]:
                key = annotation[ANNOT_FIELD_KEY][1:-1]
                if key in data_dict.keys():
                    # Makes it render when opening the pdf file on Preview for MacOs
                    # Credit @davidmacneil: https://github.com/pmaupin/pdfrw/issues/84
                    annotation.update(pdfrw.PdfDict(AP='')) 
                    # Form Field cannot be modifed Ff=1
                    annotation.update(pdfrw.PdfDict(V='{}'.format(data_dict[key]),Ff=1))
                    # Form Field can be modified without Ff=1
                    # annotation.update(pdfrw.PdfDict(V='{}'.format(data_dict[key])))
    pdfrw.PdfWriter().write(output_pdf_path, template_pdf)

### Run the function

In [35]:
if __name__ == '__main__':
    write_fillable_pdf(input_path, output_path, data_dict)